In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Writing All the functions necessery for data analysis

def normalise(data):
    data = (data - data.min()) / (data.max() - data.min())
    return data

def get_xy(data):
    data = np.array(data)
    n = len(data[0])
    x = data[:,:n-1]
    y = data[:,n-1]
    return x,y

def hold_out(x,y,test_size):
    n = len(y)
    train_index = int(n*(1-test_size))
    x_train = x[:train_index,:]
    x_test = x[train_index:,:]
    y_train = y[:train_index]
    y_test = y[train_index:]
    return x_train, y_train ,x_test , y_test

def i_of_k_fold(x,i,k):
    n = len(x)
    l = list(range(n))
    train_index = l[0:n*(i-1)//k] + l[(n*i)//k:n]
    test_index = l[n*(i-1)//k : (n*i)//k ]
    return train_index , test_index

def gradient_decent(x,y,learning_rate,iteration):
    eps = 10**-2
    n= len(x[0])
    w = [0]*n
    w0 = 0
    m = len(x)
    w_list = [ ]
    for i in range(iteration):
        y_predicted = np.dot(x,w)+w0
        dy = y_predicted - y
        dw0 = np.sum(dy)
        dw = (1/m)*np.dot(dy,x)
        w = w - learning_rate*dw
        w0 = w0 -learning_rate*dw0
    return w,w0

def linear_reg(x,y,learning_rate,iteration):
    eps = 10**-10
    n= len(x[0])
    w = [0]*n
    w0 = 0
    m = len(x)
    cost_list = [ ]
    w_list = [ ]
    for i in range(iteration):
        y_predicted = np.dot(x,w)+w0
        cost = (1/(2*m))* np.sum(np.square(y-y_predicted))
        cost_list.append(cost)
        dy = y_predicted - y
        dw0 = np.sum(dy)
        dw = (1/m)*np.dot(dy,x)
        w_old = w
        w0_old = w0
        w = w - learning_rate*dw
        w0 = w0 -learning_rate*dw0
        w_list.append(list(w)+[w0])
        if np.sum(np.square(w_old-w)) <eps:
            break
    print(f"Final Values :\n w = {w} w0 = {w0} cost = {cost} ")
    return cost_list,w_list,w,w0


## Mean error function is for determine the avarage error between predicted data and actual data.
def mean_error(x_test,y_test,w,w0):
    y_predicted = np.dot(x_test,w) + w0
    error = (1/len(x_test))*np.sum(np.square(y_predicted - y_test))
    return error

def k_fold(x_train,y_train,k):
    learning_rate_range = [10**i for i in range(-5,3)]
    mean_error_coresponding_to_LearningRate = dict()
    for lr in learning_rate_range:
        each_fold_error = [ ]
        for i in range(1,k+1):
            train_index , test_index = i_of_k_fold(x_train,i,k)
            w,w0 = gradient_decent(x_train[train_index],y_train[train_index],lr,iteration = 30)
            each_fold_error.append(mean_error(x_train[test_index],y_train[test_index],w,w0))

        mean_error_coresponding_to_LearningRate[lr] = sum(each_fold_error)/len(each_fold_error)
    return mean_error_coresponding_to_LearningRate

In [2]:
data_3 = pd.read_csv("data3.csv",header = None)

In [3]:
data_3

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,1.74920,5.35240,1.9048,3.88020,6.25810,-0.81882,0.15429,5.547400,5.03480,8.1049,...,-2.1593,-0.51192,-2.56690,-3.60630,-2.01050,1.87000,-3.94740,-3.95660,3.646400,-6.3990
1,2.56720,0.62960,6.1277,9.17090,8.20650,1.68810,-1.18850,-2.611500,10.97300,-3.3615,...,1.1644,2.45490,7.52300,-3.28330,-0.36977,-2.55980,2.06930,-4.78550,2.174800,36.5410
2,5.24510,-0.81218,4.8901,-0.61827,-0.14016,2.86660,1.31370,8.395400,1.96440,3.2947,...,4.1030,5.25320,4.43210,9.46530,-0.89669,2.97720,-1.92560,0.69556,2.621600,-9.2496
3,0.26811,2.84790,3.3763,5.97190,6.67580,-3.59950,7.57350,0.099107,0.80194,-2.5541,...,8.8078,-4.26590,4.89840,4.74880,8.08550,9.65790,2.23120,-1.70540,-1.014300,-9.6137
4,3.16490,5.86330,8.8108,6.04410,-2.84140,1.55760,7.15950,4.748200,-3.39500,-4.5881,...,-4.6686,-3.18060,4.90400,8.73860,1.34630,9.25870,0.60414,-1.87180,5.360400,6.0652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,-2.79630,2.14890,8.8594,-0.94036,4.74310,5.13220,2.75250,8.354000,-4.00710,5.2895,...,4.9511,5.35940,-0.33292,0.60967,1.55470,3.29050,10.13000,3.02670,-4.400600,30.2870
796,-4.31980,-2.28510,3.3545,5.96220,0.46250,7.91490,-3.72390,0.209850,6.11430,6.5968,...,7.3548,-2.32700,2.87490,10.03200,-3.79820,3.35680,1.29950,-1.90100,-0.004131,-1.2650
797,0.93772,-2.36160,-1.6969,7.40660,-4.56800,-2.91120,2.48260,-4.629400,5.00280,6.9308,...,1.9006,10.70300,2.34180,1.14440,7.68360,10.06300,-0.56802,-0.47637,1.081300,-36.0920
798,6.80770,9.59510,-3.2787,6.58890,1.92600,6.13950,6.70350,9.776700,0.83240,8.3612,...,-4.5515,10.86100,-1.75380,8.64750,8.94460,0.91756,-1.02130,-4.23760,-2.314900,-50.1770


## 2 Analysisng data3

In [4]:
x,y = get_xy(data_3)  ## Separate x and y value from data
x_train, y_train ,x_test , y_test = hold_out(x,y,0.2)  ## spliting into train and test data (holdout method).
##finding avarage error with respect to each learning rate
mean_error_coresponding_to_LearningRate = k_fold(x_train,y_train,10)  
mean_error_coresponding_to_LearningRate

C:\Users\adity_000\anaconda3\lib\site-packages\ipykernel_launcher.py:79: RuntimeWarning: overflow encountered in square


{1e-05: 1371.8495708660507,
 0.0001: 1234.055268895704,
 0.001: 3412042771.5432467,
 0.01: 3.718411670753657e+82,
 0.1: 3.771698631784197e+143,
 1: 4.732067490494795e+203,
 10: 4.840406929947744e+263,
 100: inf}

### learning rae = 0.0001

## Assignment

In [5]:
 cost_list_d3,w_list_d3,w_d3,w0_d3 = linear_reg(x_train,y_train,0.0008,1000)

Final Values :
 w = [ 5.71121272e-02 -2.15396847e+00 -2.56658220e-01 -3.58019521e-01
  2.68369943e-01  5.57767714e-02  5.12931089e-01  6.86746234e-01
 -2.29127575e-01 -2.20547758e-01 -7.50459847e-01 -2.42049096e-01
  4.10626004e-01  1.12239039e+00 -6.70498582e-01  2.12060648e+00
 -8.39606008e-03  9.26585087e-02  8.52811165e-04 -7.67307369e-01
  5.37881834e-01 -3.94460294e-01 -5.32878263e-02  5.46507007e-02
 -2.88913155e-01  3.84247109e-02 -1.17878634e+00  4.13611784e-02
 -9.35531384e-03  5.01423203e-03  3.29514682e-01  8.58224614e-01
 -1.35302701e-01  6.02997225e-01  1.91326624e-02  5.17558187e-01
 -2.31463325e-01  3.58282771e-01  5.41576356e-03  2.46552156e-01
 -9.40536749e-02  3.41565134e-01  1.86875224e-01  4.33121833e-01
  5.36972601e-01  5.09715055e-02  1.37952696e-01  1.44031064e-01
  1.21589226e-01 -4.42724981e-01  2.34486032e-01 -2.14656631e-01
  5.89602172e-03  2.24085978e+00 -6.27656536e-01 -3.89941837e-01
 -3.90726001e-01 -4.83989964e-02  5.66036199e-01  1.76853929e-02
  1.2

### 2(a) Find the parameters using direct method and iterative methods
#### iterative method

In [6]:
print(w0_d3,w_d3)

21.43393278595145 [ 5.71121272e-02 -2.15396847e+00 -2.56658220e-01 -3.58019521e-01
  2.68369943e-01  5.57767714e-02  5.12931089e-01  6.86746234e-01
 -2.29127575e-01 -2.20547758e-01 -7.50459847e-01 -2.42049096e-01
  4.10626004e-01  1.12239039e+00 -6.70498582e-01  2.12060648e+00
 -8.39606008e-03  9.26585087e-02  8.52811165e-04 -7.67307369e-01
  5.37881834e-01 -3.94460294e-01 -5.32878263e-02  5.46507007e-02
 -2.88913155e-01  3.84247109e-02 -1.17878634e+00  4.13611784e-02
 -9.35531384e-03  5.01423203e-03  3.29514682e-01  8.58224614e-01
 -1.35302701e-01  6.02997225e-01  1.91326624e-02  5.17558187e-01
 -2.31463325e-01  3.58282771e-01  5.41576356e-03  2.46552156e-01
 -9.40536749e-02  3.41565134e-01  1.86875224e-01  4.33121833e-01
  5.36972601e-01  5.09715055e-02  1.37952696e-01  1.44031064e-01
  1.21589226e-01 -4.42724981e-01  2.34486032e-01 -2.14656631e-01
  5.89602172e-03  2.24085978e+00 -6.27656536e-01 -3.89941837e-01
 -3.90726001e-01 -4.83989964e-02  5.66036199e-01  1.76853929e-02
  1.239

#### Direct Method

In [7]:
def direct_method(x,y):
    ### w = (XTX)−1 XTy
    x = np.insert(x,0,1, axis=1)
    a = np.linalg.inv(np.dot(x.T,x))
    w = np.dot(np.dot(a,x.T) , y)
    return w

In [8]:
w_direct = direct_method(x,y)
w_direct

array([ 2.30002142e+01,  5.25013352e-02, -2.15893730e+00, -2.59215755e-01,
       -3.61778578e-01,  2.65592920e-01,  5.53263742e-02,  5.14764694e-01,
        6.89366932e-01, -2.35092743e-01, -2.23094690e-01, -7.54443267e-01,
       -2.45496505e-01,  4.09448861e-01,  1.12218100e+00, -6.72873966e-01,
        2.12241540e+00, -1.19924132e-02,  9.13556325e-02, -3.16613415e-03,
       -7.67707975e-01,  5.34120820e-01, -4.01188058e-01, -5.73296347e-02,
        4.98451769e-02, -2.85947342e-01,  3.43589766e-02, -1.18818561e+00,
        3.71326876e-02, -1.35519303e-02,  4.04717389e-04,  3.28067889e-01,
        8.57696085e-01, -1.40253696e-01,  6.00420253e-01,  1.47755892e-02,
        5.06825306e-01, -2.36061899e-01,  3.53986311e-01,  2.20392044e-03,
        2.42541811e-01, -8.70566672e-02,  3.41987064e-01,  1.87631569e-01,
        4.29021258e-01,  5.35019685e-01,  4.97822529e-02,  1.31069126e-01,
        1.45942170e-01,  1.16821125e-01, -4.52389592e-01,  2.30166277e-01,
       -2.17620872e-01,  

### 2(c) Describe the search space and cross validation techniques used to find the values
### of the hyperparameters of the model

### 2(d) Report the values of the hyperparameters.

In [9]:
print("Errors coressponding to Hyper Parameters :")
mean_error_coresponding_to_LearningRate

Errors coressponding to Hyper Parameters :


{1e-05: 1371.8495708660507,
 0.0001: 1234.055268895704,
 0.001: 3412042771.5432467,
 0.01: 3.718411670753657e+82,
 0.1: 3.771698631784197e+143,
 1: 4.732067490494795e+203,
 10: 4.840406929947744e+263,
 100: inf}

#### Here we checked parameters values between 0.0001 and 0.001 and found 0.0008 as hyper parameter for building the model for optimal performance

### 2(e) Report the parameters of the model.

In [10]:
print('Parameters of the model is :')
print(w0_d3,w_d3)

Parameters of the model is :
21.43393278595145 [ 5.71121272e-02 -2.15396847e+00 -2.56658220e-01 -3.58019521e-01
  2.68369943e-01  5.57767714e-02  5.12931089e-01  6.86746234e-01
 -2.29127575e-01 -2.20547758e-01 -7.50459847e-01 -2.42049096e-01
  4.10626004e-01  1.12239039e+00 -6.70498582e-01  2.12060648e+00
 -8.39606008e-03  9.26585087e-02  8.52811165e-04 -7.67307369e-01
  5.37881834e-01 -3.94460294e-01 -5.32878263e-02  5.46507007e-02
 -2.88913155e-01  3.84247109e-02 -1.17878634e+00  4.13611784e-02
 -9.35531384e-03  5.01423203e-03  3.29514682e-01  8.58224614e-01
 -1.35302701e-01  6.02997225e-01  1.91326624e-02  5.17558187e-01
 -2.31463325e-01  3.58282771e-01  5.41576356e-03  2.46552156e-01
 -9.40536749e-02  3.41565134e-01  1.86875224e-01  4.33121833e-01
  5.36972601e-01  5.09715055e-02  1.37952696e-01  1.44031064e-01
  1.21589226e-01 -4.42724981e-01  2.34486032e-01 -2.14656631e-01
  5.89602172e-03  2.24085978e+00 -6.27656536e-01 -3.89941837e-01
 -3.90726001e-01 -4.83989964e-02  5.6603619

### 2(f) Compare the performance of the two models using statistical techniques.

In [11]:
error_i = mean_error(x_test,y_test,w_d3,w0_d3)
error_d = mean_error(x_test,y_test,w_direct[1:],w_direct[0])
print("MSE using itreative method : ",error_i)
print("MSE using direct method : ",error_d)

MSE using itreative method :  0.06924304306709514
MSE using direct method :  3.755111288881987e-07


### As the error is less so we can conclude direct method is better then iterative method